In [1]:
import numpy as np
import tensorflow as tf
import gym
from collections import deque

/Users/apple/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [58]:
# 하이퍼 파라미터
gridSize = 7
nbActions = gridSize * gridSize
nbStates = gridSize * gridSize

LEARNING_RATE = 0.005
INPUT = nbStates
OUTPUT = nbActions
DISCOUNT = 0.99

winReward = 1

STONE_NONE = 0
STONE_PLAYER1 = 1
STONE_PLAYER2 = -1
STONE_MAX = 5

In [59]:
hiddenSize = 100
maxMemory = 500
batchSize = 50
epoch = 100
epsilonStart = 1
epsilonDiscount = 0.999
epsilonMinimumValue = 0.1
discount = 0.9
learningRate = 0.0001
winReward = 1

In [74]:
class Omok:
    # --------------------------------
    # 초기화
    # --------------------------------
    def __init__(self, gridSize):
        self.gridSize = gridSize
        self.nbStates = self.gridSize * self.gridSize
        self.state = np.zeros(self.nbStates, dtype=np.uint8)
        
    # --------------------------------
    # 리셋
    # --------------------------------
    def reset(self):
        self.state = np.zeros(self.nbStates, dtype=np.uint8)
        
    # --------------------------------
    # 현재 상태 구함
    # --------------------------------
    def getState(self):
        return np.reshape(self.state, (1, self.nbStates))
    
    # --------------------------------
    # render
    # --------------------------------
    def render(self):
        state = np.reshape(self.state,(self.gridSize,self.gridSize))
        print(state)
        return state
    
    # --------------------------------
    # 매칭 검사
    # 다섯개 붙어 있으면 true, 아님 false
    # --------------------------------
    def CheckMatch(self, player):
        for y in range(self.gridSize):
            for x in range(self.gridSize):

                # --------------------------------
                # 오른쪽 검사
                # --------------------------------
                match = 0

                for i in range(STONE_MAX):
                    if (x + i >= self.gridSize):
                        break

                    if (self.state[y * self.gridSize + x + i] == player):
                        match += 1
                    else:
                        break;

                    if (match >= STONE_MAX):
                        return True

                # --------------------------------
                # 아래쪽 검사
                # --------------------------------
                match = 0

                for i in range(STONE_MAX):
                    if (y + i >= self.gridSize):
                        break

                    if (self.state[(y + i) * self.gridSize + x] == player):
                        match += 1
                    else:
                        break;

                    if (match >= STONE_MAX):
                        return True

                # --------------------------------
                # 오른쪽 대각선 검사
                # --------------------------------
                match = 0

                for i in range(STONE_MAX):
                    if ((x + i >= self.gridSize) or (y + i >= self.gridSize)):
                        break

                    if (self.state[(y + i) * self.gridSize + x + i] == player):
                        match += 1
                    else:
                        break;

                    if (match >= STONE_MAX):
                        return True

                # --------------------------------
                # 왼쪽 대각선 검사
                # --------------------------------
                match = 0

                for i in range(STONE_MAX):
                    if ((x - i < 0) or (y + i >= self.gridSize)):
                        break

                    if (self.state[(y + i) * self.gridSize + x - i] == player):
                        match += 1
                    else:
                        break;

                    if (match >= STONE_MAX):
                        return True

        return False
    
    # --------------------------------
    # 게임오버 검사
    # 리워드를 리턴
    # --------------------------------
    def getReward(self, player):
        if (self.CheckMatch(STONE_PLAYER1) == True):
            if (player == STONE_PLAYER1):
                return winReward
            else:
                return -winReward
        elif (self.CheckMatch(STONE_PLAYER2) == True):
            if (player == STONE_PLAYER1):
                return -winReward
            else:
                return winReward
        else:
            for i in range(self.nbStates):
                if (self.state[i] == STONE_NONE):
                    return 0
            return 0

        
    # --------------------------------
    # action취해서 state update
    # player : 다음 player로 전환
    # --------------------------------
    def getNextState(self, state, player, action):
        state[action] = player;
        self.state = state
        player = -player
        
        return self.state, player
    
    
    # --------------------------------
    # 한 에피소드 동안 실행(게임이 끝날 때까지)
    # --------------------------------
    def step(self, sess):
        trainExamples = []
        self.curPlayer = 1
        
        while True:
            #canonicalBoard = self.game.getCanonicalForm(self.state,self.curPlayer)
            pi = sess.run(output_layer, feed_dict={X: self.state, keep_prob: 1.0})
            
            trainExamples.append([self.state, self.curPlayer, pi])
            action = np.random.choice(len(pi), p=pi)
            
            self.state, self.curPlayer = self.getNextState(self.state, self.curPlayer, action)
            
            r = self.getReward(self.state, self.curPlayer)
            
            if r!=0:
                return [(x[0],x[2],r*((-1)**(x[1]!=self.curPlayer))) for x in trainExamples]
            
                # 전체 에피소드 (한 수 한 수....)
                # 각 수마다 state, policy, reward
                # 마지막에 이긴 player가 둔 수에는 모두 r=1, 진 player가 둔 수는 모두 r=-1

In [60]:
# network architecture

X = tf.placeholder(tf.float32, [None, nbStates])
x_image = tf.reshape(X, [-1, gridSize, gridSize, 1])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([3, 3, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([3, 3, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([3 * 3 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 3*3*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, nbActions])
b_fc2 = bias_variable([nbActions])

output_layer=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

Y = tf.placeholder(tf.float32, [None, nbActions])
cost = tf.reduce_sum(tf.square(Y - output_layer)) / (2 * batchSize)
optimizer = tf.train.AdamOptimizer(learningRate).minimize(cost)